## You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb).

In [ ]:
!pip install deeppavlov

## Data Preparation

In [1]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

data = SimpleDSTC2DatasetReader().read('my_data')

2019-08-15 18:54:45.171 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 269: [downloading data from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data]
2019-08-15 18:54:45.698 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data/simple_dstc2.tar.gz
100%|██████████| 497k/497k [00:00<00:00, 54.2MB/s]
2019-08-15 18:54:45.716 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_data/simple_dstc2.tar.gz archive into my_data
2019-08-15 18:54:45.741 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading dialogs from my_data/simple-dstc2-trn.json]
2019-08-15 18:54:45.810 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading dialogs from my_data/simple-dstc2-val.json]
2019-08-15 18:54:45.859 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading di

In [2]:
!ls my_data

simple-dstc2-templates.txt  simple-dstc2-tst.json
simple-dstc2-trn.json	    simple-dstc2-val.json


The input file format of each of `simple-dstc2-trn.json`, `simple-dstc2-val.json` and `simple-dstc2-tst.json` is the following:

In [3]:
!head -n 18 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },


In [4]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed DSTC2 dialogs:

In [5]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

User utterances:
----------------

[   {'prev_resp_act': None, 'text': ''},
    {   'prev_resp_act': 'welcomemsg',
        'text': 'i would like to find a restaurant in the north part of town '
                'that serves kosher food'},
    {   'db_result': {},
        'prev_resp_act': 'api_call',
        'text': 'i would like to find a restaurant in the north part of town '
                'that serves kosher food'},
    {'prev_resp_act': 'canthelp_area_food', 'text': 'italian food'},
    {   'db_result': {   'addr': '20 milton road chesterton',
                         'area': 'north',
                         'food': 'italian',
                         'name': 'da vinci pizzeria',
                         'phone': '01223 351707',
                         'postcode': 'c.b 4, 1 j.y',
                         'pricerange': 'cheap'},
        'prev_resp_act': 'api_call',
        'text': 'italian food'},
    {   'prev_resp_act': 'inform_area+inform_food+offer_name',
        'text': 'can 

## 1. Build database of items

For a valid goal-oriented bot there should be a database of relevant items: restarants' information for the task of booking a restaurant, hotels' info for hotel booking etc.

    >> database([{'pricerange': 'cheap', 'area': 'south'}])
    
    Out[1]: 
        [[{'name': 'the lucky star',
           'food': 'chinese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way cherry hinton',
           'phone': '01223 244277',
           'postcode': 'c.b 1, 7 d.y'},
          {'name': 'nandos',
           'food': 'portuguese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way',
           'phone': '01223 327908',
           'postcode': 'c.b 1, 7 d.y'}]]

In [6]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(table_name="mytable",
                           primary_keys=["name"],
                           keys=["name", "food", "pricerange", "area", "addr", "phone", "postcode"],
                           save_path="my_bot/db.sqlite")

2019-08-15 18:54:49.128 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-08-15 18:54:49.129 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 67: Initializing empty database on /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.


Let's find all db_result request results and add it to our database of restaurants:

In [7]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

2019-08-15 18:54:50.67 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 138: Created table with keys {'name': 'text', 'food': 'text', 'pricerange': 'text', 'area': 'text', 'addr': 'text', 'phone': 'text', 'postcode': 'text'}.


Adding 3016 items.


We can now play with the database and make requests to it:

In [8]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'name': 'nandos',
   'food': 'portuguese',
   'pricerange': 'cheap',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way',
   'phone': '01223 327908',
   'postcode': 'c.b 1, 7 d.y'},
  {'name': 'the lucky star',
   'food': 'chinese',
   'pricerange': 'cheap',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way cherry hinton',
   'phone': '01223 244277',
   'postcode': 'c.b 1, 7 d.y'}]]

In [9]:
!ls my_bot

db.sqlite


## 2. Build Slot Filler

Slot Filler is component that inputs text and outputs dictionary of slot names and their values:

    >> slot_filler(['I would like some chineese food'])
    
    Out[1]: [{'food': 'chinese'}]

To implement a slot filler you need to provide
    
 - **slot types**
 - all possible **slot values**
 - optionally, it will be good to provide examples of mentions for every value of each slot
 
The data should be in `slot_vals.json` file with the following format:

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                

Let's use a simple non-trainable slot filler that relies on levenshtein distance:

### Non-trainable slot filler

In [10]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2019-08-15 18:54:52.686 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 9.24MB/s]
2019-08-15 18:54:52.688 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [11]:
!ls my_bot/slotfill

dstc_slot_vals.json


In [12]:
!head -n 10 my_bot/slotfill/dstc_slot_vals.json

{
    "food": {
        "caribbean": [
            "carraibean",
            "carribean",
            "caribbean"
        ],
        "kosher": [
            "kosher"
        ],


Let's check performance of our slot filler:

In [13]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json


slotfill_raw_config = read_json(configs.ner.slotfill_dstc2_raw)

In [14]:
slotfill_raw_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_raw_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

In [15]:
from deeppavlov import evaluate_model

slotfill_raw = evaluate_model(slotfill_raw_config);

2019-08-15 18:54:55.258 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 95: [downloading data from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/vimary/code-projects/Pilot/examples/my_data]
2019-08-15 18:54:55.287 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/vimary/code-projects/Pilot/examples/my_data/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 55.1MB/s]
2019-08-15 18:54:55.299 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting /home/vimary/code-projects/Pilot/examples/my_data/dstc2_v2.tar.gz archive into /home/vimary/code-projects/Pilot/examples/my_data
2019-08-15 18:54:55.329 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-15 18:54:55.469 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2

{"valid": {"eval_examples_count": 1444, "metrics": {"slots_accuracy": 0.9307}, "time_spent": "0:00:27"}}
{"test": {"eval_examples_count": 1386, "metrics": {"slots_accuracy": 0.9481}, "time_spent": "0:00:26"}}


We've got slot accuracy of **93% on valid** set and **94% on test** set.

Let's interact with it:

In [16]:
from deeppavlov import build_model

slotfill_raw = build_model(slotfill_raw_config)

In [17]:
slotfill_raw(['i want cheap chinee food'])

[{'food': 'chinese', 'pricerange': 'cheap'}]

Saving slotfill config file to disk (we will require it's path later):

In [18]:
import json

json.dump(slotfill_raw_config, open('my_bot/slotfill_raw_config.json', 'wt'))

In [19]:
!ls my_bot

db.sqlite  slotfill  slotfill_raw_config.json


## Train bot

In [20]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_dstc2)

**Configure** bot to use our built **database**:

In [21]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'table_name': 'mytable',
    'primary_keys': ["name"],
    'keys': ["name", "food", "pricerange", "area", "addr", "phone", "postcode"],
    'save_path': 'my_bot/db.sqlite'
}

**Configure** bot to use levenshtein distance based **slot filler**:

In [22]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_raw_config.json'

gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = \
    ['pricerange', 'this', 'area', 'food']

**Configure** bot to use your **response templates**.

Templates should be in the format `<act>TAB<template>`, where `<act>` is action label and `<template>` is a corresponding text response. Text response might contain slot type names, every `#slot_type` will be filled with the slot value from the dialogue state.

In [23]:
!head my_data/simple-dstc2-templates.txt -n 5

api_call	api_call area="#area" food="#food" pricerange="#pricerange"
bye	You are welcome!
canthear	Sorry, I can't hear you.
canthelp_area	I'm sorry but there is no #area american restaurant in the #area of town.
canthelp_area_food	Sorry there is no #food restaurant in the #area of town.


In [24]:
gobot_config['chainer']['pipe'][-1]['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['template_path'] = 'my_data/simple-dstc2-templates.txt'

You can use a simple **bag-of-words as embedder** (by default):

In [25]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

Specify train/valid/test **data path** and **path to save** the final bot model:

In [26]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

##### Training policy network

In [27]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 8 # set batch size
gobot_config['train']['max_batches'] = 200 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 50 # evaluate on full 'valid' split each 30 batches
gobot_config['train']['log_every_n_batches'] = 50
gobot_config['train']['log_on_k_batches'] = 30 # evaluate on 30 batches of 'train' split every 50 batches

train_model(gobot_config);

2019-08-15 18:56:34.627 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-15 18:56:34.821 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-08-15 18:56:34.969 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
2019-08-15 18:56:35.274 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 98: [saving vocabulary to /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
W0815 18:56:36.276948 140357696087680 deprecation_wrapper.py:119] From /home/vimary/code-projects/Pilot/deeppavlov/core/models/tf_model.py:38: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0815 18:56:36.27

2019-08-15 18:56:38.410 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 251: [initializing `GoalOrientedBot` from scratch]
I0815 18:56:38.410912 140357696087680 network.py:251] [initializing `GoalOrientedBot` from scratch]
2019-08-15 18:57:58.671 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.0175
I0815 18:57:58.671278 140357696087680 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.0175
2019-08-15 18:57:58.672 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0815 18:57:58.672056 140357696087680 nn_trainer.py:166] Saving model
2019-08-15 18:57:58.672 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 18:57:58.672759 140357696087680 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
W0815 18:57:58.673794 140357696087680 deprecation_w

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0175}, "time_spent": "0:01:21", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 240, "metrics": {"per_item_dialog_accuracy": 0.4657}, "time_spent": "0:02:38", "epochs_done": 0, "batches_seen": 50, "train_examples_seen": 400, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.4682220101356507}}


2019-08-15 19:00:35.395 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.3343
I0815 19:00:35.395404 140357696087680 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.3343
2019-08-15 19:00:35.396 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0815 19:00:35.396290 140357696087680 nn_trainer.py:166] Saving model
2019-08-15 19:00:35.397 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 19:00:35.397149 140357696087680 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-08-15 19:00:35.465 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 764: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0815 19:00:35.465843 140357696087680 network.py:764] [saving parameters to /home/vimary/code-projects/Pilot/examp

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.3343}, "time_spent": "0:03:57", "epochs_done": 0, "batches_seen": 50, "train_examples_seen": 400, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 240, "metrics": {"per_item_dialog_accuracy": 0.5442}, "time_spent": "0:05:15", "epochs_done": 0, "batches_seen": 100, "train_examples_seen": 800, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.7673896300792694}}


...

2019-08-15 19:06:03.712 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 248: [initializing `GoalOrientedBot` from saved]
I0815 19:06:03.712948 140357696087680 network.py:248] [initializing `GoalOrientedBot` from saved]
2019-08-15 19:06:03.714 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 770: [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0815 19:06:03.714329 140357696087680 network.py:770] [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
2019-08-15 19:06:03.717 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 19:06:03.717086 140357696087680 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4134}, "time_spent": "0:01:24"}}


2019-08-15 19:08:45.161 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
I0815 19:08:45.161855 140357696087680 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-08-15 19:08:45.163 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
W0815 19:08:45.163285 140357696087680 serializable.py:47] No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-08-15 19:08:45.164 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 63: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
I0815 19:08:45.164417 140357696087680 sqlite_database.py:63] Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-08-15 19:08:45.248 IN

{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.408}, "time_spent": "0:01:18"}}


2019-08-15 19:08:45.896 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 248: [initializing `GoalOrientedBot` from saved]
I0815 19:08:45.896820 140357696087680 network.py:248] [initializing `GoalOrientedBot` from saved]
2019-08-15 19:08:45.897 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 770: [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0815 19:08:45.897950 140357696087680 network.py:770] [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
2019-08-15 19:08:45.899 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0815 19:08:45.899963 140357696087680 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]


See [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) for advanced configuration of train process.

##### Metric scores on valid&test

Calculating **accuracy** of trained bot: whether predicted system responses match true responses' texts.

In [ ]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

2019-08-15 19:08:46.825 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
I0815 19:08:46.825568 140357696087680 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-15 19:08:47.96 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
I0815 19:08:47.096684 140357696087680 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-08-15 19:08:47.342 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
I0815 19:08:47.342166 140357696087680 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4134}, "time_spent": "0:01:21"}}


With settings of `max_batches=200`, valid accuracy `~ 0.4` and test accuracy is `0.4`.

##### Chatting with bot

In [28]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2019-08-15 19:08:45.938 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
I0815 19:08:45.938536 140357696087680 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-08-15 19:08:45.940 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
W0815 19:08:45.940229 140357696087680 serializable.py:47] No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-08-15 19:08:45.941 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 63: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
I0815 19:08:45.941222 140357696087680 sqlite_database.py:63] Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-08-15 19:08:46.28 INF

In [29]:
bot(['hi, want chinee food, can you suggest a place to go?'])

2019-08-15 19:08:46.629 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 505: Made api_call with {'food': 'chinese'}, got 16 results.
I0815 19:08:46.629126 140357696087680 network.py:505] Made api_call with {'food': 'chinese'}, got 16 results.


['Rice house serves chinese food.']

In [30]:
bot(['ok, i want ther address'])

['Sure, rice house is on 88 mill road city centre.']

In [31]:
bot(['now i want french food in the center of town'])

2019-08-15 19:08:46.724 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 505: Made api_call with {'food': 'french', 'area': 'centre'}, got 1 results.
I0815 19:08:46.724561 140357696087680 network.py:505] Made api_call with {'food': 'french', 'area': 'centre'}, got 1 results.


['Cote is a nice place in the centre of town serving tasty french food.']

In [32]:
bot(['thanks, give me their address too'])

['Sure, cote is on bridge street city centre.']

In [33]:
bot(['and post code'])

['The post code of cote is c.b 2, 1 u.f.']

In [34]:
bot(['bye'])

['You are welcome!']

In [35]:
bot.reset()